In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point



In [3]:
df_trips = pd.read_csv("data/trips.txt")
df_routes = pd.read_csv("data/routes.txt")
df_stops = pd.read_csv("data/stops.txt")
df_stop_times = pd.read_csv("data/stop_times.txt")
df_pathways = pd.read_csv("data/pathways.txt")

## Járatok

In [4]:
df_trips

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,8140,C0418310,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_5,Y701,1.0,1.0
1,8140,C0418311,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_6,Y702,1.0,1.0
2,8140,C0418312,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_7,Y701,1.0,1.0
3,8140,C0418313,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_8,Y702,1.0,1.0
4,8140,C0418314,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_9,Y701,1.0,1.0
...,...,...,...,...,...,...,...,...,...
276457,H9,H55125_43,59635,Csömör,0,NaN,43,NaN,1.0
276458,H9,H55126_43,59642,Csömör,0,NaN,43,NaN,1.0
276459,H9,H55174_44,59642,Örs vezér tere,1,NaN,44,NaN,1.0
276460,H9,H55176_44,59606,Örs vezér tere,1,NaN,44,NaN,1.0


## Melyik járat, melyik megállóban mikor áll meg?

In [5]:
df_stop_times


,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,C0418310,F04181,08:03:00,08:03:00,0,NaN,NaN,NaN,0.0
1,C0418310,F04526,08:06:00,08:06:00,1,NaN,NaN,NaN,160.0
2,C0418311,F04526,08:30:00,08:30:00,0,NaN,NaN,NaN,0.0
3,C0418311,F04181,08:33:00,08:33:00,1,NaN,NaN,NaN,160.0
4,C0418312,F04181,08:33:00,08:33:00,0,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...
5752577,H800027_1,09191212,24:40:00,24:40:00,13,NaN,NaN,NaN,13725.0
5752578,H800027_1,09209214,24:42:00,24:42:00,14,NaN,NaN,NaN,14558.0
5752579,H800027_1,09134218,24:44:00,24:45:00,15,NaN,NaN,NaN,16348.0
5752580,H800027_1,09217219,24:49:00,24:49:00,16,NaN,NaN,NaN,19548.0


In [6]:
df_stops


,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN
3,004716,"ÉD metró járműtelep,porta",47.469651,19.129090,004716,NaN,NaN,NaN,2.0
4,004948,Metró ÉD járműtelep (kapu),47.465239,19.142612,004948,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6189,MERGED_F01401,Közvágóhíd H,47.467527,19.075886,F01401,1.0,stop,NaN,2.0
6190,MERGED_F02865,Nagy Lajos király útja / Czobor utca,47.522827,19.102034,F02865,1.0,stop,NaN,2.0
6191,MERGED_F03146,"Újpalota, Szentmihályi út",47.541663,19.151091,F03146,1.0,stop,NaN,1.0
6192,MERGED_F03810,Ferihegy vasútállomás,47.436741,19.222565,F03810,1.0,stop,NaN,2.0


## Útvonalak

Példa: Keresünk útvonalszámokat:

In [7]:
df_routes[ df_routes["route_short_name"].isin(["6","7","9"])]

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_color,route_text_color,route_sort_order
1,BKK,0070,7,NaN,3,"Albertfalva vasútállomás / Újpalota, Nyírpalot...",009EE3,FFFFFF,23
5,BKK,0090,9,NaN,3,"Óbuda, Bogdáni út / Kőbánya alsó vasútállomás",009EE3,FFFFFF,27
259,BKK,3060,6,NaN,0,Széll Kálmán tér M / Móricz Zsigmond körtér M,FFD800,000000,21
371,BKK,VP06,6,NaN,3,Móricz Zsigmond körtér M / Széll Kálmán tér M,009EE3,FFFFFF,22


## Keressük meg egy útvonalon a megállók neveit

Megoldási terv: Keresek egy routet-t és az első trip-et benne.

Ahhoz a triphez lekérdezzük a megállókat.
Utána majd mellétesszük a neveket is



In [8]:
df_trips.groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})

,route_id,trip_id,trip_headsign
route_id,,,
0050,0050,D0166810,Pasaréti tér
0070,0070,D06142101,"Újpalota, Nyírpalota út"
0075,0075,D0172110,Blaha Lujza tér M
0078,0078,C755691,"Újpalota, Nyírpalota út"
0085,0085,D0166110,Kelenföld vasútállomás M
...,...,...,...
VP41,VP41,D056341087,Kamaraerdei Ifj. Park
VP50,VP50,D064891,"Pestszentlőrinc, Béke tér"
VP52,VP52,D048871,"P.erzsébet, Pacsirtatelep"


### Melyik megállókban áll meg a 4-6-os ?

#### Mikor indul reggel a 4-6-os

A 3060-as járat első indulását keresem.
A groupby és agg funckiók miatt fontos

In [35]:
df_trips[ df_trips["route_id"].isin(["3060"]) ].groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})["trip_id"]

route_id
3060    D015151403
Name: trip_id, dtype: object

Megvan a trip_id

In [36]:
df_stop_times[ df_stop_times["trip_id"] == "D015151066"]

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
1428865,D015151066,F01998,05:47:00,05:47:00,0,Széll Kálmán tér M,NaN,NaN,0.0
1428866,D015151066,F01992,05:49:00,05:49:00,1,Széll Kálmán tér M,NaN,NaN,544.0
1428867,D015151066,F02225,05:50:00,05:50:00,2,Széll Kálmán tér M,NaN,NaN,1087.0
1428868,D015151066,F01374,05:51:00,05:51:00,3,Széll Kálmán tér M,NaN,NaN,1750.0
1428869,D015151066,F01380,05:53:00,05:53:00,4,Széll Kálmán tér M,NaN,NaN,2073.0
1428870,D015151066,F01191,05:55:00,05:55:00,5,Széll Kálmán tér M,NaN,NaN,2395.0
1428871,D015151066,F01199,05:57:00,05:57:00,6,Széll Kálmán tér M,NaN,NaN,2948.0
1428872,D015151066,F01200,05:58:00,05:58:00,7,Széll Kálmán tér M,NaN,NaN,3233.0
1428873,D015151066,F01168,06:00:00,06:00:00,8,Széll Kálmán tér M,NaN,NaN,3677.0
1428874,D015151066,F01111,06:01:00,06:01:00,9,Széll Kálmán tér M,NaN,NaN,4109.0


### Most tegyük mellé a neveket is!

A hátsó stop_name oszlopban látszik

In [63]:
pd.merge( df_stop_times[ df_stop_times["trip_id"] == "D015151066"], df_stops, on="stop_id")

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
0,D015151066,F01998,05:47:00,05:47:00,0,Széll Kálmán tér M,NaN,NaN,0.0,Újbuda-központ M,47.474042,19.046862,F01998,NaN,NaN,NaN,1.0
1,D015151066,F01992,05:49:00,05:49:00,1,Széll Kálmán tér M,NaN,NaN,544.0,Budafoki út / Szerémi sor,47.474114,19.053747,F01992,NaN,NaN,NaN,1.0
2,D015151066,F02225,05:50:00,05:50:00,2,Széll Kálmán tér M,NaN,NaN,1087.0,"Petőfi híd, budai hídfő",47.476956,19.059572,F02225,NaN,NaN,NaN,1.0
3,D015151066,F01374,05:51:00,05:51:00,3,Széll Kálmán tér M,NaN,NaN,1750.0,Boráros tér H,47.480400,19.066697,F01374,NaN,NaN,NaN,2.0
4,D015151066,F01380,05:53:00,05:53:00,4,Széll Kálmán tér M,NaN,NaN,2073.0,Mester utca / Ferenc körút,47.482775,19.068848,F01380,NaN,NaN,NaN,1.0
5,D015151066,F01191,05:55:00,05:55:00,5,Széll Kálmán tér M,NaN,NaN,2395.0,Corvin-negyed M,47.485500,19.069924,F01191,NaN,NaN,NaN,1.0
6,D015151066,F01199,05:57:00,05:57:00,6,Széll Kálmán tér M,NaN,NaN,2948.0,Harminckettesek tere,47.490357,19.070841,F01199,NaN,NaN,NaN,1.0
7,D015151066,F01200,05:58:00,05:58:00,7,Széll Kálmán tér M,NaN,NaN,3233.0,Rákóczi tér M,47.492866,19.071187,F01200,NaN,NaN,NaN,1.0
8,D015151066,F01168,06:00:00,06:00:00,8,Széll Kálmán tér M,NaN,NaN,3677.0,Blaha Lujza tér M,47.496770,19.070725,F01168,NaN,NaN,NaN,1.0
9,D015151066,F01111,06:01:00,06:01:00,9,Széll Kálmán tér M,NaN,NaN,4109.0,Wesselényi utca / Erzsébet körút,47.500350,19.068934,F01111,NaN,NaN,NaN,1.0


# Kódrészlet ami egy vonal megállóit keresi meg
Most egy kis kódrészlet keresi nekem az adott vonal megállóit.
Ezt lehet majd beletenni egy függvénybe

In [51]:
#route_id = "3060"
#route_id="3040"
route_id="0090"
elso_trip = df_trips[ df_trips["route_id"] == route_id ].groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})["trip_id"].iloc[0]
a=pd.merge( df_stop_times[ df_stop_times["trip_id"] == elso_trip], df_stops, on="stop_id")
a["route_id"]=route_id
a[["route_id","trip_id","stop_id","stop_name","arrival_time","departure_time","stop_lat","stop_lon"]]

,route_id,trip_id,stop_id,stop_name,arrival_time,departure_time,stop_lat,stop_lon
0,0090,D033981412,F01636,Kőbánya alsó vasútállomás,04:36:00,04:36:00,47.483652,19.127839
1,0090,D033981412,F01653,Szent László tér,04:37:00,04:37:00,47.485925,19.131093
2,0090,D033981412,F01641,Liget tér,04:39:00,04:39:00,47.482795,19.130358
3,0090,D033981412,F01571,Kőbánya alsó vasútállomás (Mázsa tér),04:40:00,04:40:00,47.482879,19.125715
4,0090,D033981412,F01563,Egészségház,04:42:00,04:42:00,47.484126,19.116668
5,0090,D033981412,F01733,Eiffel Műhelyház,04:43:00,04:43:00,47.485136,19.112914
6,0090,D033981412,F01267,Kőbányai út / Könyves Kálmán körút,04:45:00,04:45:00,47.486740,19.106971
7,0090,D033981412,008036,Kőbányai út 31.,04:45:00,04:45:00,47.487461,19.104252
8,0090,D033981412,F01187,Orczy tér,04:47:00,04:48:00,47.489470,19.090495
9,0090,D033981412,F01223,Kálvária tér,04:49:00,04:49:00,47.488890,19.085146


## Melyik járatok állnak meg ebben a megállóban ?

In [52]:
df_stop_times[ df_stop_times["stop_id"] == "048506"]

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
548787,C96610106,048506,07:27:00,07:27:00,9,Zugló vasútállomás,NaN,NaN,3713.0
548839,C96610111,048506,07:37:00,07:37:00,9,Zugló vasútállomás,NaN,NaN,3713.0
548889,C96610116,048506,07:44:00,07:44:00,9,Zugló vasútállomás,NaN,NaN,3713.0
548941,C96610121,048506,07:52:00,07:52:00,9,Zugló vasútállomás,NaN,NaN,3713.0
548966,C966101221,048506,16:35:00,16:35:00,9,Zugló vasútállomás,NaN,NaN,3713.0
...,...,...,...,...,...,...,...,...,...
5688598,D066221107,048506,03:09:00,03:09:00,42,"Rákospalota, Székely Elek út",NaN,NaN,20957.0
5688676,D066221115,048506,03:30:00,03:30:00,42,"Rákospalota, Kossuth utca",NaN,NaN,20957.0
5688727,D066221125,048506,03:50:00,03:50:00,26,NaN,NaN,NaN,13366.0
5688784,D066221131,048506,04:09:00,04:09:00,42,Újpest-központ M,NaN,NaN,20957.0


Látszik, hogy jó sok indulás van ebből a megállóból

## Melyik útvonalak állnak meg ebben a megállóban?


In [64]:
stop_id = "048506" # Szent István Bazilika
#stop_id = "F01029" # Nyugati
parent_stop_id = df_stops[ df_stops["stop_id"] == stop_id]["parent_station"].iloc[0]
parent_stop_id

nan

In [65]:
if parent_stop_id is not np.nan:
        stop_id = parent_stop_id
stop_name = df_stops[ df_stops["stop_id"] == stop_id]["stop_name"].iloc[0]
stop_name

'Szent István Bazilika'

Itt látszik, hogy ez igy nem lesz OK.

In [55]:
df_megallo_jaratok = pd.merge( df_stop_times[ df_stop_times["stop_id"] == stop_id], df_trips).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","stop_headsign","stop_sequence"]]


,stop_id,stop_name,route_id,stop_headsign,stop_sequence
0,F01029,Nyugati pályaudvar M,9230,NaN,42
10,F01029,Nyugati pályaudvar M,4730,Keleti pályaudvar M ► Kápolna tér,1
65,F01029,Nyugati pályaudvar M,9140,"Káposztásmegyer, Mogyoródi-patak",33
79,F01029,Nyugati pályaudvar M,9310,Nyugati pályaudvar M ► Zöldmál,5
84,F01029,Nyugati pályaudvar M,9312,Zöldmál,0
117,F01029,Nyugati pályaudvar M,VP06,Széll Kálmán tér M,9
163,F01029,Nyugati pályaudvar M,9141,Újpest-központ M,15
270,F01029,Nyugati pályaudvar M,9500,"Rákospalota, Székely Elek út",45
271,F01029,Nyugati pályaudvar M,9501,NaN,45
418,F01029,Nyugati pályaudvar M,0090,"Óbuda, Bogdáni út",20


Ez sajnos nagyon nem OK. Itt több más járatot is jó lenne látni.

## Mi a nyugati főmegállója?

Hát vagy ez vagy nincs kötlve a táblázat rendesen

In [56]:
stop_id = "F00933"
stop_name = df_stops[ df_stops["stop_id"] == stop_id]["stop_name"].iloc[0]
stop_name


'Nyugati pályaudvar M'

In [57]:
stop_id = "F01029"
stop_name = df_stops[ df_stops["stop_id"] == stop_id]["stop_name"].iloc[0]
stop_name


'Nyugati pályaudvar M'

In [58]:
stop_id = "F00933"
stop_id = "F01029"

parent_stop_id = df_stops[ df_stops["stop_id"] == stop_id]["parent_station"].iloc[0]
parent_stop_id

nan

Melyek azok az állomások amiknek van főmegállója?

In [59]:
df_stops[  df_stops["parent_station"] != np.nan ]
df_stops[  df_stops["parent_station"] != "NaN" ]
df_stops[  df_stops["parent_station"].notna() ]


,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
202,008359,Óbudai autóbuszgarázs,47.567996,19.026289,F00678,NaN,merged,MERGED_F00678,1.0
203,008365,Bécsi út / Vörösvári út,47.548489,19.028994,F00568,NaN,merged,MERGED_F00568,1.0
266,008545,Puskás Ferenc Stadion M,47.500105,19.107013,008545,NaN,NaN,008875,1.0
445,009085,Budaörs vasútállomás,47.448166,18.964870,009085,NaN,NaN,008690,1.0
665,009630,"Budakalász, Auchan áruház",47.611510,19.062341,009630,NaN,NaN,008822,1.0
...,...,...,...,...,...,...,...,...,...
6174,LMXKVC,Kálvin tér [C],47.489727,19.062364,LMXKVC,2.0,NaN,CS056227,2.0
6175,LMXKVD,Kálvin tér [D],47.489226,19.061814,LMXKVD,2.0,NaN,CS056227,2.0
6176,LMXKVE,Kálvin tér [E],47.489559,19.061525,LMXKVE,2.0,NaN,CS056227,2.0
6177,LMXKVF,Kálvin tér [F],47.489696,19.061327,LMXKVF,2.0,NaN,CS056227,2.0


In [60]:
df_stops[ df_stops["parent_station"] == "F01029"]

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding


In [61]:
df_stops[ df_stops["parent_station"] == "008875"]

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
266,008545,Puskás Ferenc Stadion M,47.500105,19.107013,008545,NaN,NaN,008875,1.0


In [62]:
df_stops[ df_stops["parent_station"] == "CS056227"]

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
941,056227,Kálvin tér,47.488903,19.061022,056227,NaN,NaN,CS056227,1.0
942,056228,Kálvin tér,47.489245,19.061938,056228,NaN,NaN,CS056227,1.0
2811,F01289,Kálvin tér,47.490170,19.060640,F01289,NaN,NaN,CS056227,1.0
2812,F01290,Kálvin tér,47.489490,19.061920,F01290,NaN,NaN,CS056227,1.0
6055,LM3KV1,Kálvin tér,47.489693,19.061707,LM3KV1,3.0,NaN,CS056227,1.0
6133,LM4KV1,Kálvin tér,47.489048,19.061394,LM4KV1,3.0,NaN,CS056227,2.0
6171,LMXKV1,Kálvin tér (lift),47.489010,19.061593,LMXKV1,2.0,NaN,CS056227,1.0
6172,LMXKVA,Kálvin tér [A],47.489987,19.061298,LMXKVA,2.0,NaN,CS056227,2.0
6173,LMXKVB,Kálvin tér [B],47.489919,19.062093,LMXKVB,2.0,NaN,CS056227,2.0
6174,LMXKVC,Kálvin tér [C],47.489727,19.062364,LMXKVC,2.0,NaN,CS056227,2.0


# KÖZELI ÁTSZÁLLÁSI PONTOK

Ez úgy tűnik földrajzi koordináták alapján kell megkeresni.

In [25]:

# Create a list of Point objects
points = [Point(xy) for xy in zip(df_stops['stop_lon'], df_stops['stop_lat'])]

# Create a GeoSeries from the list of points with the correct CRS
gs = gpd.GeoSeries(points, crs='EPSG:4326')

gdf_stops = gpd.GeoDataFrame(df_stops, geometry=gs, crs="EPSG:4326")
gdf_stops.head(5)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN,POINT (19.1357 47.50037)
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0,POINT (19.12789 47.48314)
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN,POINT (19.10341 47.50037)
3,004716,"ÉD metró járműtelep,porta",47.469651,19.129090,004716,NaN,NaN,NaN,2.0,POINT (19.12909 47.46965)
4,004948,Metró ÉD járműtelep (kapu),47.465239,19.142612,004948,NaN,NaN,NaN,NaN,POINT (19.14261 47.46524)


Az EPSG:4326 nem jó távolságmérésre, mert nem tudom miért.
De a 3035 az jó, Németországhoz kötött valami. Ezért átmegyek arra:

In [26]:
gdf_stops.to_crs("epsg:3035",inplace=True)
gdf_stops.head(5)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN,POINT (5007781.301 2752100.781)
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0,POINT (5007426.257 2750125.866)
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN,POINT (5005367.638 2751801.574)
3,004716,"ÉD metró járműtelep,porta",47.469651,19.129090,004716,NaN,NaN,NaN,2.0,POINT (5007694.861 2748647.499)
4,004948,Metró ÉD járműtelep (kapu),47.465239,19.142612,004948,NaN,NaN,NaN,NaN,POINT (5008764.678 2748285.946)


Most a Nyugatihoz viszonyítva fogunk közeli megállókat keresni

In [27]:
stop_id = "F01029" # Nyugati
gdf_nyugati = gdf_stops [ gdf_stops["stop_id"] == stop_id ]
gdf_megallo = gdf_nyugati.to_crs(epsg=3035)
gdf_megallo

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
2579,F01029,Nyugati pályaudvar M,47.509806,19.057151,F01029,NaN,NaN,NaN,1.0,POINT (5001786.02 2752416.904)


In [28]:
gdf_nearest_stops = gdf_megallo.sjoin_nearest(gdf_stops,distance_col="distance", how="right", exclusive = True, max_distance = 500)
gdf_nearest_stops.columns
#gdf_nearest_stops[["stop_id_left","stop_name_left","stop_lat_left","stop_lon_left","stop_id_right","stop_name_right","stop_lat_right","stop_lon_right","distance"]]


Index(['index_left', 'stop_id_left', 'stop_name_left', 'stop_lat_left',
       'stop_lon_left', 'stop_code_left', 'location_type_left',
       'location_sub_type_left', 'parent_station_left',
       'wheelchair_boarding_left', 'stop_id_right', 'stop_name_right',
       'stop_lat_right', 'stop_lon_right', 'stop_code_right',
       'location_type_right', 'location_sub_type_right',
       'parent_station_right', 'wheelchair_boarding_right', 'geometry',
       'distance'],
      dtype='object')

In [29]:
gdf_nearest_stops.nsmallest(40,'distance')[["stop_id_left","stop_name_left","stop_lat_left","stop_lon_left","stop_id_right","stop_name_right","stop_lat_right","stop_lon_right","distance"]]

,stop_id_left,stop_name_left,stop_lat_left,stop_lon_left,stop_id_right,stop_name_right,stop_lat_right,stop_lon_right,distance
2503,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00935,Nyugati pályaudvar M,47.509922,19.056658,39.253332
6099,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYI,Nyugati pályaudvar [I],47.509870,19.056404,56.669957
2502,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00934,Nyugati pályaudvar M,47.509896,19.056394,57.835671
6095,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYE,Nyugati pályaudvar [E],47.510447,19.056426,89.626772
6098,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYH,Nyugati pályaudvar [H],47.510233,19.056103,91.944253
2501,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00933,Nyugati pályaudvar M,47.510300,19.056167,92.071854
6097,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYG,Nyugati pályaudvar [G],47.510326,19.056196,92.107868
6090,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NY3,Nyugati pályaudvar (lift » M3),47.510040,19.055784,106.075445
129,F01029,Nyugati pályaudvar M,47.509806,19.057151,008137,Nyugati pályaudvar,47.510571,19.056072,117.418620
6096,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYF,Nyugati pályaudvar [F],47.510703,19.056257,120.137919


In [30]:
stop_id = "F01029"
df_pathways


,pathway_id,pathway_mode,is_bidirectional,from_stop_id,to_stop_id,traversal_time
0,LM4KF1-LM4KFA,1,1,LM4KF1,LM4KFA,90
1,LM4KF1-LM4KFB,2,1,LM4KF1,LM4KFB,180
2,LM3UK2-F00913,5,1,LM3UK2,F00913,90
3,LM4KF1-LM4KFC,2,1,LM4KF1,LM4KFC,210
4,LM4KF1-LM4KFD,2,1,LM4KF1,LM4KFD,180
...,...,...,...,...,...,...
488,LM3NT1-F01252-WC,5,1,LM3NT1,F01252,60
489,LM3AHF-LM3AH1,2,1,LM3AHF,LM3AH1,180
490,LM3EU1-F01494,5,1,LM3EU1,F01494,90
491,L008165-F00898,2,1,L008165,F00898,30


In [31]:
stop_list = df_pathways [ df_pathways ["from_stop_id"] == stop_id]
stop_list

,pathway_id,pathway_mode,is_bidirectional,from_stop_id,to_stop_id,traversal_time
